In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/estudiantes/estudiantes.csv


# Cargar los datos

In [4]:
df = pd.read_csv('/kaggle/input/estudiantes/estudiantes.csv')

df.head()
#print(df.shape)

,escuela,sexo,edad,direccion,tamano_familia,estatus_padres,educacion_madre,educacion_padre,trabajo_madre,trabajo_padre,...,calidad_relaciones_familiares,tiempo_libre,salidas,consume_alc_entre_semana,consume_alc_fin_semana,salud,ausencias,P1,P2,P3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


# Preparar los datos

In [ ]:
#Eliminar filas con valores nulos, esto es opcional ya que mejor lo remplazamos por ceros
#df= df.dropna()

#verificar que los tipos de datos de cada columna sean correcto
#tipos_de_dato = df.dtypes
#print(tipos_de_dato)

#Hacer que la codificacion este correcta para que no haya errores de caracteres.
# Convertir todas las celdas del DataFrame a UTF-8
#data_frame = data_frame.applymap(lambda x: x.encode('latin-1').decode('utf-8', 'ignore'))

# Encontrar registros con todos los valores duplicados
#duplicados_completos = df[df.duplicated(df.columns, keep=False)]

#Imprimir los registros completos duplicados si los hay
#if not duplicados_completos.empty:
  #  print("SI HAY:")
#else:
  #  print("No hay registros con todos los valores iguales en el DataFrame.")
    
# Eliminar registros duplicados del DataFrame
#data_frame_sin_duplicados = data_frame.drop_duplicates()
# Imprimir información sobre la cantidad de registros eliminados
#registros_duplicados_elim = len(data_frame) - len(data_frame_sin_duplicados)
#print("Se eliminaron {} registros duplicados del DataFrame.".format(registros_duplicados_elim))
#data_frame = data_frame_sin_duplicados

#Remplazar valores nulos de registros por 0
#data_frame.fillna(0, inplace=True)
#Verificar si hay alguna celda nula en el DataFrame
#hay_nulos = df.isnull().any()
#if hay_nulos.any():
 #   print("Hay al menos una celda nula en el DataFrame.")
#else:
 #   print("No hay celdas nulas en el DataFrame.")

# Agregar columna estatus
esta columna marca un reprobado si en algun parcial el alumno saco menos del minimo aprobatorio (14)

In [33]:
#hola mundo
# Añadir la columna 'estatus'
df['estatus'] = df.apply(lambda row: '1' if row['P1'] < 14 or row['P2'] < 14 or row['P3'] < 14 else '0', axis=1)

Cantidad de estudiantes aprobados: 73
Cantidad de estudiantes reprobados: 322


# Modelo

In [40]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Input, Dense # type: ignore
from tensorflow.keras.models import Model # type: ignore

epocas = 50
entrenamiento = 0.70
size = [13, 6]

entradas = ['direccion', 'tamano_familia', 'estatus_padres', 'educacion_madre', 'educacion_padre','trabajo_madre','trabajo_padre','razon_escuela','traslado_escuela','tiempo_estudio','reprobacion','apoyo_educativo_adicional','apoyo_educativo_familiar','clases_extra_pagas','actividades_extracurriculares','asistio_guarderia','quiere_educacion_superior','internet','en_relacion_romantica','calidad_relaciones_familiares','tiempo_libre','salidas','consume_alc_entre_semana','consume_alc_fin_semana','salud','ausencias']

# Codificar las columnas categóricas a valores numéricos
label_encoders = {}
for column in entradas:
    if df[column].dtype == 'object':
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])
        label_encoders[column] = le

# Preparar los datos de entrada y salida
datos_entrada = df[entradas].values
datos_salida = df[['P1', 'P2', 'P3', 'estatus']].values


# Dividir los datos en Training set y Validation set
cantidad_datos = len(df)
entrenamiento = int(cantidad_datos * entrenamiento)

training_input = datos_entrada[:entrenamiento]
training_output = datos_salida[:entrenamiento]
validation_input = datos_entrada[entrenamiento:]
validation_output = datos_salida[entrenamiento:]


# Convertir los valores a tensores
train_inputs = tf.convert_to_tensor(training_input, dtype=tf.float32)
train_outputs_continuos = tf.convert_to_tensor(training_output[:, :3], dtype=tf.float32)
train_outputs_binario = tf.convert_to_tensor(training_output[:, 3], dtype=tf.float32)
val_inputs = tf.convert_to_tensor(validation_input, dtype=tf.float32)
val_outputs_continuos = tf.convert_to_tensor(validation_output[:, :3], dtype=tf.float32)
val_outputs_binario = tf.convert_to_tensor(validation_output[:, 3], dtype=tf.float32)

# Definir el modelo
input_layer = Input(shape=(len(entradas),))
hidden_layer = Dense(4, activation='relu')(input_layer)
output_continuos = Dense(3, activation='linear')(hidden_layer)  # Tres salidas lineales
output_binario = Dense(1, activation='sigmoid')(hidden_layer)  # Una salida sigmoide

model = Model(inputs=input_layer, outputs=[output_continuos, output_binario])

# Compilar el modelo
model.compile(optimizer='adam', loss=['mean_squared_error', 'binary_crossentropy'])

# Entrenar el modelo con datos de entrenamiento y validación
history = model.fit(train_inputs, [train_outputs_continuos, train_outputs_binario], epochs=epocas, validation_split=0.2, batch_size=32, verbose=1)

# Evaluar el modelo con los datos de validación
loss = model.evaluate(val_inputs, [val_outputs_continuos, val_outputs_binario])
print(f'Loss en el conjunto de validación: {loss}')

# Predecir con nuevas entradas
predicciones_continuos, predicciones_binario = model.predict(val_inputs)

# Aplicar umbral a las predicciones binarias
predicciones_binario = (predicciones_binario > 0.5).astype(int)

# Crear una figura y dos subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(size[0], size[1]))

# Obtener la pérdida del historial de entrenamiento
loss = history.history['loss']

# Graficar la pérdida en el primer subplot
epochs = range(1, len(loss) + 1)
ax1.plot(epochs, loss, 'b', label='Pérdida de entrenamiento')
ax1.set_title('Pérdida durante entrenamiento del modelo')
ax1.set_xlabel('Épocas')
ax1.set_ylabel('Pérdida')
ax1.legend()

# Graficar las salidas esperadas y las predicciones en el segundo subplot
for i in range(3):  # Iterar sobre las tres salidas continuas (P1, P2, P3)
    ax2.plot(range(len(validation_output)), validation_output[:, i], 'bo-', label=f'Salida esperada {i+1}' if i == 0 else "")
    ax2.plot(range(len(predicciones_continuos)), predicciones_continuos[:, i], 'ro-', label=f'Predicciones {i+1}' if i == 0 else "")

ax2.plot(range(len(validation_output)), validation_output[:, 3], 'go-', label='Estatus esperado')
ax2.plot(range(len(predicciones_binario)), predicciones_binario, 'mo-', label='Predicción estatus')

ax2.set_title('Salidas esperadas y predicciones')
ax2.set_xlabel('Muestras')
ax2.set_ylabel('Valor')
ax2.legend()

# Ajustar los subplots para evitar solapamiento
plt.tight_layout()

# Mostrar la figura con los dos subplots
plt.show()

# Función para predecir con nuevos datos
def predecir_nuevo(direccion, tamano_familia, estatus_padres, educacion_madre, educacion_padre, trabajo_madre, trabajo_padre,
                   razon_escuela, traslado_escuela, tiempo_estudio, reprobacion, apoyo_educativo_adicional, apoyo_educativo_familiar,
                   clases_extra_pagas, actividades_extracurriculares, asistio_guarderia, quiere_educacion_superior, internet,
                   en_relacion_romantica, calidad_relaciones_familiares, tiempo_libre, salidas, consume_alc_entre_semana,
                   consume_alc_fin_semana, salud, ausencias):

    nuevo_dato = np.array([[direccion, tamano_familia, estatus_padres, educacion_madre, educacion_padre, trabajo_madre,
                            trabajo_padre, razon_escuela, traslado_escuela, tiempo_estudio, reprobacion, apoyo_educativo_adicional,
                            apoyo_educativo_familiar, clases_extra_pagas, actividades_extracurriculares, asistio_guarderia,
                            quiere_educacion_superior, internet, en_relacion_romantica, calidad_relaciones_familiares, tiempo_libre,
                            salidas, consume_alc_entre_semana, consume_alc_fin_semana, salud, ausencias]])

    # Normalizar el nuevo dato
    for i, column in enumerate(entradas):
        if column in label_encoders:
            nuevo_dato[:, i] = label_encoders[column].transform(nuevo_dato[:, i])
        max_valor = df[column].max()
        min_valor = df[column].min()
        nuevo_dato[:, i] = nuevo_dato[:, i] / abs(max_valor - min_valor)
    
    # Predecir con el nuevo dato
    prediccion_continuos, prediccion_binario = model.predict(nuevo_dato)
    prediccion_binario = (prediccion_binario > 0.5).astype(int)
    return prediccion_continuos, prediccion_binario

# Ejemplo de predicción con un nuevo dato (puedes cambiar los valores según tu interés)
nueva_prediccion_continuos, nueva_prediccion_binario = predecir_nuevo(1, 2, 1, 3, 4, 2, 3, 1, 2, 2, 0, 0, 1, 0, 1, 0, 1, 1, 0, 4, 2, 3, 2, 3, 4, 6)
print('parciales:', nueva_prediccion_continuos)
print('estatus:', nueva_prediccion_binario)

Epoch 1/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 132.1919 - val_loss: 120.3770
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 128.7842 - val_loss: 118.4918
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 120.4873 - val_loss: 116.5915
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 125.6526 - val_loss: 114.5462
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 123.1730 - val_loss: 112.3572
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 117.7610 - val_loss: 110.0169
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 114.1079 - val_loss: 107.5216
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 116.5543 - val_loss: 104.8131
Epoch 9/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 108.6542 - val_loss: 101.8414
Epoch 10/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 104.1369 - val_loss: 98.3917
Epoch 11/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 106.6346 - val_loss: 94.3953
Epoch 12/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - lo